<a name="about"></a>
About this notebook
======

This notebook assumes you have ran the local Iris notebook and you have not deleted the LOCAL_ROOT folder. In this notebook, we will use online prediction on a pre-trained Tensorflow model. This notebook will does not assume that the notebook "3. Iris Classification Cloud Training" was executed.

<a name="setup"></a>
Setting things up
=====

In [13]:
import datalab_solutions.structured_data as sd

Lets look at the versions of structured_data and TF we have. Make sure TF is 1.0.0, and SD is 0.0.1.

In [14]:
import os
import pandas as pd

import tensorflow as tf
from tensorflow.python.lib.io import file_io

print('tf ' + str(tf.__version__))
print('sd ' + str(sd.__version__))

tf 1.0.0
sd 0.0.1


This notebook will write files during prediction. Please give a root folder you wish to use.

In [15]:
LOCAL_ROOT = './iris_notebook_workspace' # This should be the same as what was used in the local iris notebook
CLOUD_ROOT = 'gs://' + datalab_project_id() + 'iris-classification-datalab'

if not file_io.file_exists(LOCAL_ROOT):
  raise ValueError('LOCAL_ROOT not found. Did you run the local notebook?')
!gsutil mb {CLOUD_ROOT}

Creating gs://cloud-ml-deviris-classification-datalab/...
ServiceException: 409 Bucket cloud-ml-deviris-classification-datalab already exists.


First, let us put the csv files on GCS and the output of training.

In [16]:
!gsutil -m cp {os.path.join(LOCAL_ROOT, '*.csv')} {CLOUD_ROOT}
!gsutil -m cp -r {os.path.join(LOCAL_ROOT, 'training')} {CLOUD_ROOT}

Copying file://./iris_notebook_workspace/eval.csv [Content-Type=text/csv]...
Copying file://./iris_notebook_workspace/train.csv [Content-Type=text/csv]...
Copying file://./iris_notebook_workspace/predict.csv [Content-Type=text/csv]...
/ [3/3 files][  5.3 KiB/  5.3 KiB] 100% Done                                    
Operation completed over 3 objects/5.3 KiB.                                      
Copying file://./iris_notebook_workspace/training/model/variables/variables.data-00000-of-00001 [Content-Type=application/octet-stream]...
Copying file://./iris_notebook_workspace/training/model/assets.extra/transforms.json [Content-Type=application/json]...
Copying file://./iris_notebook_workspace/training/evaluation_model/saved_model.pb [Content-Type=application/octet-stream]...
Copying file://./iris_notebook_workspace/training/model/saved_model.pb [Content-Type=application/octet-stream]...
Copying file://./iris_notebook_workspace/training/model/assets.extra/schema.json [Content-Type=applicati

In [17]:
!gsutil ls {CLOUD_ROOT}/training

gs://cloud-ml-deviris-classification-datalab/training/evaluation_model/
gs://cloud-ml-deviris-classification-datalab/training/model/
gs://cloud-ml-deviris-classification-datalab/training/staging/
gs://cloud-ml-deviris-classification-datalab/training/train/


Online Prediction using ML Engine
==========

We first have to deploy the model, and then we can make requests on it.

In [18]:
!gcloud beta ml models create iris_model --regions=us-central1
!gcloud beta ml versions create v1 --model iris_model --origin {CLOUD_ROOT}/training/model

Creating version (this might take a few minutes)......done.


In [19]:
sd.cloud_predict(
  model_name='iris_model',
  model_version='v1',
  data=['101,6.3,3.3,6,2.5', #virginica
        '100,5.7,2.8,4.1,1.3', #versicolor
        '13,4.8,3,1.4,0.1', #setosa
  ]
)

,key_from_input,top_1_label,top_1_score,top_2_label,top_2_score,top_3_label,top_3_score
0,101,Iris-virginica,0.952918,Iris-versicolor,0.047047,Iris-setosa,3.49441e-05
1,100,Iris-versicolor,0.811353,Iris-virginica,0.174092,Iris-setosa,0.014555
2,13,Iris-setosa,0.953729,Iris-versicolor,0.0462122,Iris-virginica,5.8815e-05


In [20]:
sd.cloud_predict(
  model_name='iris_model',
  model_version='v1',
  data=pd.DataFrame(
    [[101,6.3,3.3,6,2.5],
     [100,5.7,2.8,4.1,1.3],
     [13,4.8,3,1.4,0.1]])
)

,key_from_input,top_1_label,top_1_score,top_2_label,top_2_score,top_3_label,top_3_score
0,101,Iris-virginica,0.952918,Iris-versicolor,0.047047,Iris-setosa,3.49441e-05
1,100,Iris-versicolor,0.811353,Iris-virginica,0.174092,Iris-setosa,0.014555
2,13,Iris-setosa,0.953729,Iris-versicolor,0.0462122,Iris-virginica,5.8815e-05


Cleaning things up
=====

If you want to delete the files you made on GCS, uncomment and run the next cell.

In [21]:
#!gsutil rm -fr {CLOUD_ROOT}

To delete the models created, uncomment and run the next cell.

In [22]:
#!gcloud beta ml versions delete v1 --model iris_model 
#!gcloud beta ml models delete iris_model 